# 🎯 Cebu Pacific Customer Support Agent Optimization

**Complete DSPy Agent Optimization with MLflow Tracking**

This notebook demonstrates automated AI agent optimization using:
- **DSPy Framework**: Prompt optimization
- **Groq LLM**: llama-3.1-8b-instant (fast inference)
- **MIPROv2 Optimizer**: Automatic instruction generation
- **MLflow**: Experiment tracking and logging

## Demonstration Flow:

### **STEP 1: Show the Problem**
  → Display real messy support ticket
  → Run unoptimized agent (generic, unhelpful response)
  → Metrics: Time = 5 min, Quality = 30%

### **STEP 2: Show DSPy Optimization**
  → Load 50 past tickets with good resolutions
  → Run MIPROv2 optimization (3-5 minutes)
  → Track with MLflow
  → Show optimized instructions + few-shot examples

### **STEP 3: Show the Results**
  → Re-run same ticket with optimized agent
  → Detailed, step-by-step solution
  → Metrics: Time = 30 sec, Quality = 85%

### **STEP 4: Show Business Impact**
  → Calculate: 1000 tickets/day × 4.5 min saved = 75 hours/day
  → 75 hours × $30/hour = $2,250/day = $821K/year savings
  → ROI: ♾️ (optimization costs ~$1)

**Technology Stack:**
- DSPy with MIPROv2
- Groq llama-3.1-8b-instant
- MLflow experiment tracking
- Custom evaluation metrics

**Dataset:**
- 50 training examples (successful resolutions)
- 20 validation examples (test scenarios)
- Real Cebu Pacific support tickets

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 2: Install and Import Required Packages
# ═══════════════════════════════════════════════════════════════════════════

# Install required packages (run once)
import sys

# Note: Run this in terminal first if packages not installed:
# pip install dspy-ai mlflow matplotlib numpy python-dotenv

try:
    import dspy
    import mlflow
    import matplotlib.pyplot as plt
    print("✓ All packages already installed!")
except ImportError:
    print("Installing required packages...")
    !{sys.executable} -m pip install -q dspy-ai mlflow matplotlib numpy python-dotenv
    import dspy
    import mlflow
    import matplotlib.pyplot as plt
    print("✓ Packages installed successfully!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# CELL 3: Import All Required Libraries
# ═══════════════════════════════════════════════════════════════════════════

import os
import json
import time
import numpy as np
from typing import List, Dict, Any
from datetime import datetime
from functools import lru_cache

print("✅ All libraries imported successfully!")
print(f"   DSPy version: {dspy.__version__ if hasattr(dspy, '__version__') else 'Unknown'}")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Python version: {sys.version.split()[0]}")